In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig
warnings.filterwarnings('ignore')

torch.manual_seed(52)
np.random.seed(52)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [48]:
# Define the amino acid alphabet
AMINO_ACIDS = 'ACDEFGHIKLMNPQRSTVWY'
AA_TO_IDX = {aa: idx for idx, aa in enumerate(AMINO_ACIDS)}

class ProteinDataset(Dataset):
    """Custom dataset class for protein sequences and their properties."""
    
    def __init__(self, sequences, targets, target_name, max_length=500):
        """
        Initialize the dataset.
        
        Args:
            sequences: List of protein sequences as strings
            targets: List of target values
            target_name: Name of the target property being predicted
            max_length: Maximum sequence length for padding/truncation
        """
        self.sequences = sequences
        self.targets = targets
        self.target_name = target_name
        self.max_length = max_length
        
        # Remove any rows with missing values
        valid_indices = []
        for i, (seq, target) in enumerate(zip(sequences, targets)):
            if pd.notna(seq) and pd.notna(target) and len(seq) > 0:
                valid_indices.append(i)
        
        self.sequences = [sequences[i] for i in valid_indices]
        self.targets = [targets[i] for i in valid_indices]
        
        # Normalize targets using z-score normalization
        self.target_scaler = StandardScaler()
        self.normalized_targets = self.target_scaler.fit_transform(
            np.array(self.targets).reshape(-1, 1)
        ).flatten()
        
        print(f"Dataset initialized with {len(self.sequences)} valid samples")
        print(f"Target property: {target_name}")
        print(f"Target range: {min(self.targets):.3f} to {max(self.targets):.3f}")
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        """Get a single sequence-target pair."""
        sequence = self.sequences[idx]
        target = self.normalized_targets[idx]
        
        # Convert sequence to integer indices
        sequence_tensor = self.sequence_to_indices(sequence)
        target_tensor = torch.tensor(target, dtype=torch.float32)
        
        return sequence_tensor, target_tensor
    
    def sequence_to_indices(self, sequence):
        """Convert a protein sequence string to integer indices for embedding."""
        # Clean sequence - remove any non-amino acid characters
        cleaned_sequence = ''.join([aa for aa in sequence.upper() if aa in AMINO_ACIDS])
        
        # Truncate if too long
        if len(cleaned_sequence) > self.max_length:
            cleaned_sequence = cleaned_sequence[:self.max_length]
        
        # Convert to indices (0 is reserved for padding)
        indices = [AA_TO_IDX[aa] + 1 for aa in cleaned_sequence]  # +1 to reserve 0 for padding
        
        # Pad with zeros to max_length
        while len(indices) < self.max_length:
            indices.append(0)  # 0 is padding token
        
        return torch.tensor(indices, dtype=torch.long)
    
    def denormalize_target(self, normalized_value):
        """Convert normalized target back to original scale."""
        return self.target_scaler.inverse_transform([[normalized_value]])[0][0]

In [49]:
class ProteinLSTM(nn.Module):
    """LSTM-based neural network for protein sequence analysis with embedding layers."""
    
    def __init__(self, vocab_size=21, embed_dim=128, hidden_size=256, num_layers=3, dropout_rate=0.5):
        """
        Initialize the LSTM architecture with embedding.
        
        Args:
            vocab_size: Size of vocabulary (21 for 20 amino acids + padding token)
            embed_dim: Dimension of embedding vectors
            hidden_size: Number of hidden units in LSTM layers
            num_layers: Number of LSTM layers
            dropout_rate: Dropout probability for regularization
        """
        super(ProteinLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embed_dim = embed_dim
        
        # Embedding layer to convert amino acid indices to dense vectors
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        
        # Add positional encoding to help the model understand sequence positions
        self.pos_encoding = nn.Parameter(torch.randn(1, 500, embed_dim) * 0.1)
        
        # Input projection layer
        self.input_projection = nn.Linear(embed_dim, hidden_size // 2)
        
        # Bidirectional LSTM layers
        self.lstm = nn.LSTM(
            input_size=hidden_size // 2,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0,
            bidirectional=True
        )
        
        # Attention mechanism for sequence-level representation
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size * 2,  # *2 because bidirectional
            num_heads=8,
            dropout=dropout_rate,
            batch_first=True
        )
        
        # Output layers
        self.fc1 = nn.Linear(hidden_size * 2, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
        
        self.dropout = nn.Dropout(dropout_rate)
        self.layer_norm = nn.LayerNorm(hidden_size * 2)
        
        # Initialize embeddings with Xavier uniform
        nn.init.xavier_uniform_(self.embedding.weight)
        # Set padding embedding to zero
        with torch.no_grad():
            self.embedding.weight[0].fill_(0)
        
    def forward(self, x):
        """
        Forward pass through the network.
        
        Args:
            x: Input tensor of shape [batch_size, sequence_length] with amino acid indices
            
        Returns:
            predictions: Tensor of shape [batch_size, 1] with predicted values
        """
        batch_size, seq_length = x.size()
        
        # Create mask for padding tokens
        mask = (x != 0).float()  # 1 for real tokens, 0 for padding
        
        # Convert indices to embeddings
        x = self.embedding(x)  # [batch_size, seq_length, embed_dim]
        
        # Add positional encoding
        if seq_length <= self.pos_encoding.size(1):
            x = x + self.pos_encoding[:, :seq_length, :]
        
        # Apply mask to embeddings
        x = x * mask.unsqueeze(-1)
        
        # Project input to hidden dimension
        x = F.relu(self.input_projection(x))
        
        # LSTM processing
        lstm_out, (hidden, cell) = self.lstm(x)
        
        # Apply mask to LSTM output
        lstm_out = lstm_out * mask.unsqueeze(-1)
        
        # Apply layer normalization
        lstm_out = self.layer_norm(lstm_out)
        
        # Self-attention to get sequence-level representation
        # Create attention mask for padding
        attn_mask = mask.unsqueeze(1).expand(-1, seq_length, -1)
        attn_mask = attn_mask * attn_mask.transpose(1, 2)
        attn_mask = attn_mask.bool()
        
        attn_out, attn_weights = self.attention(
            lstm_out, lstm_out, lstm_out, 
            key_padding_mask=~mask.bool()
        )
        
        # Apply mask to attention output
        attn_out = attn_out * mask.unsqueeze(-1)
        
        # Global pooling with masking
        # Calculate sequence lengths for proper averaging
        seq_lengths = mask.sum(dim=1, keepdim=True)  # [batch_size, 1]
        
        # Mean pooling (sum and divide by actual sequence length)
        mean_pool = (attn_out * mask.unsqueeze(-1)).sum(dim=1) / seq_lengths.unsqueeze(-1)
        
        # Max pooling
        attn_out_masked = attn_out.masked_fill(~mask.unsqueeze(-1).bool(), float('-inf'))
        max_pool = torch.max(attn_out_masked, dim=1)[0]
        
        # Combine pooled representations
        x = mean_pool + max_pool
        
        # Output layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        
        x = self.fc3(x)
        
        return x

In [50]:
def train_model(model, train_loader, val_loader, num_epochs=100, learning_rate=0.001):
    """Train the LSTM model."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)
    
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_model_state = None
    
    print(f"Training on {device}")
    print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        
        for batch_idx, (sequences, targets) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
            sequences, targets = sequences.to(device), targets.to(device)
            
            optimizer.zero_grad()
            predictions = model(sequences).squeeze()
            loss = criterion(predictions, targets)
            loss.backward()
            
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for sequences, targets in val_loader:
                sequences, targets = sequences.to(device), targets.to(device)
                predictions = model(sequences).squeeze()
                loss = criterion(predictions, targets)
                val_loss += loss.item()
        
        # Calculate average losses
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        
        # Learning rate scheduling
        scheduler.step(avg_val_loss)
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict().copy()
        
        # Print progress
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")
            print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
            print("-" * 50)
    
    # Load best model
    model.load_state_dict(best_model_state)
    
    return model, train_losses, val_losses

In [6]:
def load_data(csv_file, target_property):
    """Load data from CSV file."""
    print(f"Loading data from {csv_file}...")
    
    # Read CSV file
    df = pd.read_csv(csv_file)
    
    print(f"Loaded {len(df)} rows")
    print(f"Columns: {list(df.columns)}")
    
    # Check if target property exists
    if target_property not in df.columns:
        raise ValueError(f"Target property '{target_property}' not found in CSV. Available columns: {list(df.columns)}")
    
    # Extract sequences and targets
    sequences = df['sequence'].tolist()
    targets = df[target_property].tolist()
    
    return sequences, targets

In [7]:
def evaluate_model(model, test_loader, dataset, target_property):
    """Evaluate the trained model."""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    
    predictions = []
    true_values = []
    
    with torch.no_grad():
        for sequences, targets in test_loader:
            sequences, targets = sequences.to(device), targets.to(device)
            batch_predictions = model(sequences).squeeze()
            
            predictions.extend(batch_predictions.cpu().numpy())
            true_values.extend(targets.cpu().numpy())
    
    # Convert back to original scale
    predictions = np.array(predictions)
    true_values = np.array(true_values)
    
    # Denormalize
    predictions_denorm = [dataset.denormalize_target(pred) for pred in predictions]
    true_values_denorm = [dataset.denormalize_target(true) for true in true_values]
    
    # Calculate metrics
    mse = mean_squared_error(true_values_denorm, predictions_denorm)
    mae = mean_absolute_error(true_values_denorm, predictions_denorm)
    r2 = r2_score(true_values_denorm, predictions_denorm)
    
    print(f"\nModel Evaluation Results for {target_property}:")
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"Mean Absolute Error: {mae:.4f}")
    print(f"R² Score: {r2:.4f}")
    
    return predictions_denorm, true_values_denorm, mse, mae, r2

In [8]:
def plot_results(train_losses, val_losses, predictions, true_values, target_property):
    """Plot training results and predictions."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot training curves
    ax1.plot(train_losses, label='Training Loss')
    ax1.plot(val_losses, label='Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    ax1.grid(True)
    
    # Plot predictions vs true values
    ax2.scatter(true_values, predictions, alpha=0.7)
    ax2.plot([min(true_values), max(true_values)], [min(true_values), max(true_values)], 'r--', lw=2)
    ax2.set_xlabel(f'True {target_property}')
    ax2.set_ylabel(f'Predicted {target_property}')
    ax2.set_title(f'Predictions vs True Values\n{target_property}')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

In [9]:
from fpgen.prop_prediction.metrics import get_regression_metrics, get_classification_metrics

In [63]:
def preproc(data):
    processed = []
    for line in data:
        clean_line = line.replace('\n', ' ').strip('[]')
        numbers = np.fromstring(clean_line, sep=' ')
        processed.append(numbers.tolist())
    return np.array(processed)

In [52]:
"""Main training pipeline."""
# Configuration
CSV_FILE = 'dataset_embedd.csv'  # Update this path if needed
TARGET_PROPERTY = 'em_max'  # Change this to predict different properties

# Available properties from your dataset:
# brightness, ex_max, em_max, ext_coeff, lifetime, maturation, pka, stokes_shift, qy, agg, switch_type

# Hyperparameters
MAX_LENGTH = 238
BATCH_SIZE = 32
NUM_EPOCHS = 200
LEARNING_RATE = 0.001

# Load data
sequences, targets = load_data(CSV_FILE, TARGET_PROPERTY)

# Split data
train_sequences, test_sequences, train_targets, test_targets = train_test_split(
        sequences, targets, test_size=0.2, random_state=42
    )
    
train_sequences, val_sequences, train_targets, val_targets = train_test_split(
    train_sequences, train_targets, test_size=0.2, random_state=42
)

print(f"Data split: {len(train_sequences)} train, {len(val_sequences)} validation, {len(test_sequences)} test")

# Create datasets
train_dataset = ProteinDataset(train_sequences, train_targets, TARGET_PROPERTY, MAX_LENGTH)
val_dataset = ProteinDataset(val_sequences, val_targets, TARGET_PROPERTY, MAX_LENGTH)
test_dataset = ProteinDataset(test_sequences, test_targets, TARGET_PROPERTY, MAX_LENGTH)
    
    # Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Initialize model
model = ProteinLSTM(
        vocab_size=21,  # 20 amino acids + 1 padding token
        embed_dim=128,
        hidden_size=256,
        num_layers=3,
        dropout_rate=0.5
    )

# Train model
trained_model, train_losses, val_losses = train_model(
    model, train_loader, val_loader, NUM_EPOCHS, LEARNING_RATE
)

predictions, true_values, mse, mae, r2 = evaluate_model(
    trained_model, test_loader, test_dataset, TARGET_PROPERTY
)

# Plot results
plot_results(train_losses, val_losses, predictions1, true_values, TARGET_PROPERTY)

# Save model
torch.save(trained_model.state_dict(), f'lstm_model_{TARGET_PROPERTY}_3.pth')
print(f"\nModel saved as 'lstm_model_{TARGET_PROPERTY}.pth'")


Loading data from dataset_embedd.csv...
Loaded 980 rows
Columns: ['sequence', 'brightness', 'em_max', 'ex_max', 'ext_coeff', 'lifetime', 'maturation', 'pka', 'stokes_shift', 'qy', 'agg', 'switch_type']
Data split: 627 train, 157 validation, 196 test
Dataset initialized with 519 valid samples
Target property: em_max
Target range: 382.000 to 1000.000
Dataset initialized with 124 valid samples
Target property: em_max
Target range: 424.000 to 720.000
Dataset initialized with 167 valid samples
Target property: em_max
Target range: 414.000 to 719.000
Training on cuda
Number of parameters: 5,473,537


Epoch 10/200: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:01<00:00, 11.00it/s]


Epoch 10/200
Train Loss: 0.9795, Val Loss: 0.9980
Learning Rate: 0.001000
--------------------------------------------------


Epoch 20/200: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:01<00:00, 10.94it/s]


Epoch 20/200
Train Loss: 0.9829, Val Loss: 0.9982
Learning Rate: 0.001000
--------------------------------------------------


...

Epoch 190/200: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:01<00:00, 11.01it/s]


Epoch 190/200
Train Loss: 0.9935, Val Loss: 0.9981
Learning Rate: 0.000000
--------------------------------------------------


Epoch 200/200: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:01<00:00, 10.95it/s]


Epoch 200/200
Train Loss: 0.9848, Val Loss: 0.9981
Learning Rate: 0.000000
--------------------------------------------------


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (167,) + inhomogeneous part.

In [1]:
from fpgen.prop_prediction.dataset import FPbase
dataset = FPbase('dataset.csv')
dataset.to_train_dataframe().head()

,sequence,brightness,em_max,ex_max,ext_coeff,lifetime,maturation,pka,stokes_shift,qy,agg,switch_type
558,MVSKGEELFTGVVPILVEMDGDVNGRKFSVRGVGEGDATHGKLTLK...,-0.516789,-1.357357,-1.875798,-0.814071,NaN,NaN,0.323540,0.923046,-0.056729,m,b
149,MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKF...,-0.802832,-0.408006,-0.214689,-1.192834,NaN,NaN,1.878962,-0.403015,-0.465539,NaN,b
184,MRSSKNVIKEFMRFKVRMEGTVNGHEFEIEGEGEGRPYEGHNTVKL...,-1.040228,0.883734,0.758032,-0.257845,NaN,NaN,NaN,0.074367,-1.725418,NaN,b
291,MSKGEELFTGIVPVLIELDGDVHGHKFSVRGEGEGDADYGKLEIKF...,NaN,-0.516948,-0.184759,NaN,NaN,NaN,NaN,-0.641706,NaN,m,b
30,MALSKQEIKKEMTMDYVMDGCVNGHSFTVKGDGAGKPYEGHQRLSL...,NaN,-0.610327,-0.244619,NaN,NaN,NaN,NaN,-0.694749,NaN,t,b


In [58]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model.eval()

predictions = []
true_values = []

with torch.no_grad():
    for sequences, targets in test_loader:
        sequences, targets = sequences.to(device), targets.to(device)
        batch_predictions = model(sequences).squeeze()

        predictions.extend(batch_predictions.cpu().numpy())
        true_values.extend(targets.cpu().numpy())

    # Convert back to original scale
print("Prediction shapes:", [np.array(p).shape for p in predictions])
predictions = np.array(predictions)
true_values = np.array(true_values)

zv = get_regression_metrics(
    dataset.rescale_targets(predictions, TARGET_PROPERTY),
    dataset.rescale_targets(true_values, TARGET_PROPERTY)
)
print(f'\t RMSE: {zv["rmse"]}')
print(f'\t MAE: {zv["mae"]}')
print(f'\t R2: {zv["r2"]}')
print(f'\t MAE (med.): {zv["mae_median"]}')

Prediction shapes: [(32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), (32,), 

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (167,) + inhomogeneous part.

In [70]:
CSV_FILE = 'dataset.csv'  # Update this path if needed
TARGET_PROPERTY = 'em_max'  # Change this to predict different properties

# Load data
sequences, targets = load_data(CSV_FILE, TARGET_PROPERTY)
protein = ESMProtein(sequences[1])
client = ESMC.from_pretrained("esmc_300m").to("cuda") # or "cpu"
protein_tensor = client.encode(protein)
logits_output = client.logits(
   protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
)
print(logits_output.logits, logits_output.embeddings, logits_output.embeddings.shape, len(sequences[1]))

Loading data from dataset.csv...
Loaded 980 rows
Columns: ['sequence', 'brightness', 'em_max', 'ex_max', 'ext_coeff', 'lifetime', 'maturation', 'pka', 'stokes_shift', 'qy', 'agg', 'switch_type']
ForwardTrackData(sequence=tensor([[[-39.2500, -39.2500, -39.5000,  ..., -39.2500, -39.5000, -39.5000],
         [-38.5000, -38.5000, -38.5000,  ..., -38.5000, -38.5000, -38.5000],
         [-41.2500, -41.2500, -41.2500,  ..., -41.2500, -41.2500, -41.2500],
         ...,
         [-38.7500, -38.7500, -38.7500,  ..., -38.7500, -38.7500, -38.7500],
         [-36.5000, -36.5000, -36.5000,  ..., -36.5000, -36.5000, -36.5000],
         [-35.5000, -35.5000, -35.5000,  ..., -35.5000, -35.5000, -35.5000]]],
       device='cuda:0', dtype=torch.bfloat16), structure=None, secondary_structure=None, sasa=None, function=None) tensor([[[ 0.0097, -0.0044,  0.0015,  ...,  0.0047, -0.0040, -0.0087],
         [-0.0054,  0.0091,  0.0352,  ...,  0.0341,  0.0125,  0.0333],
         [-0.0214, -0.0059,  0.0173,  ...,  